# Inputting only s11 and freq and outputting all geometry parameters

### Regular TensorFlow 

In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib import rc

In [34]:
# Import data and preprocess
df = pd.read_csv("../test_data/new leaky wave/S11_Data_combined.csv")
# df = pd.read_csv("../test_data/Grounded CPW Leaky Wave antenna/S11 Data.csv")
df = df.drop(df[df['dB(S(1,1)) []'] > 0].index) # Remove all rows with positive s11

In [35]:


# # Split into x and y
# input_x = df.drop(columns=['dB(S(1,1)) []'], axis=1)
# input_y = df[['dB(S(1,1)) []']]

# Split data into training and testing
train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)
training_cols = ['dB(S(1,1)) []', 'Freq [GHz]']
X_train = train_df[training_cols]
X_test = test_df[training_cols]
y_train = train_df.drop(columns=training_cols)
y_test = test_df.drop(columns=training_cols)

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # Initialize scaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
X_train

,"dB(S(1,1)) []",Freq [GHz]
5093,-11.918126,14.87
16636,-11.969567,17.48
6753,-7.234607,18.83
3429,-3.807548,19.64
11184,-12.744185,17.66
...,...,...
3690,-14.002540,15.86
12877,-24.370989,15.50
13288,-18.820857,16.13
2751,-10.719175,13.16


In [38]:
y_train

,cpw_in [mm],feed_l [mm],patch_l [mm],cpw_g [mm],Feed_W [mm],ground_w [mm],patch_ground_w [mm],patch_w [mm]
5093,1.50,3.25,3.75,0.18,1.0,1.00,1.0,4.75
16636,2.00,3.75,3.50,0.18,1.0,0.75,0.5,4.75
6753,1.75,4.00,3.75,0.18,1.0,1.00,1.0,4.75
3429,2.25,3.50,3.50,0.18,1.0,1.00,1.0,4.75
11184,1.50,3.75,4.25,0.18,1.0,1.00,1.0,4.75
...,...,...,...,...,...,...,...,...
3690,1.75,3.75,3.50,0.18,1.0,1.00,1.0,4.75
12877,2.00,3.75,3.50,0.12,1.0,1.00,1.0,4.75
13288,2.00,3.75,3.50,0.12,2.0,1.00,1.0,4.75
2751,2.00,3.25,3.50,0.18,1.0,1.00,1.0,4.75


In [39]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

inputs = Input(shape=(2,))
hidden1 = Dense(160, activation='relu')(inputs)
hidden2 = Dense(160, activation='relu')(hidden1)
hidden3 = Dense(160, activation='relu')(hidden2)
hidden4 = Dense(160, activation='relu')(hidden3)

outputs = Dense(8, activation='linear')(hidden4)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 160)               480       
                                                                 
 dense_3 (Dense)             (None, 160)               25760     
                                                                 
 dense_4 (Dense)             (None, 160)               25760     
                                                                 
 dense_5 (Dense)             (None, 160)               25760     
                                                                 
 dense_6 (Dense)             (None, 8)                 1288      
                                                                 
Total params: 79048 (308.78 KB)
Trainable params: 79048 (308.

In [40]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/150


475/475 [==============================] - 2s 2ms/step - loss: 0.2915 - accuracy: 0.9575 - val_loss: 0.0443 - val_accuracy: 1.0000
Epoch 2/150
475/475 [==============================] - 1s 2ms/step - loss: 0.0454 - accuracy: 1.0000 - val_loss: 0.0474 - val_accuracy: 1.0000
Epoch 3/150
475/475 [==============================] - 1s 2ms/step - loss: 0.0452 - accuracy: 1.0000 - val_loss: 0.0467 - val_accuracy: 1.0000
Epoch 4/150
475/475 [==============================] - 1s 2ms/step - loss: 0.0455 - accuracy: 1.0000 - val_loss: 0.0446 - val_accuracy: 1.0000
Epoch 5/150
475/475 [==============================] - 1s 2ms/step - loss: 0.0449 - accuracy: 1.0000 - val_loss: 0.0436 - val_accuracy: 1.0000
Epoch 6/150
475/475 [==============================] - 1s 2ms/step - loss: 0.0449 - accuracy: 1.0000 - val_loss: 0.0496 - val_accuracy: 1.0000
Epoch 7/150
475/475 [==============================] - 1s 2ms/step - loss: 0.0444 - accuracy: 1.0000 - val_loss: 0.0438 - val_accuracy: 1.0000
Epoch 8/150

In [41]:
predictions = model.predict(X_test_scaled)

  1/149 [..............................] - ETA: 5s

149/149 [==============================] - 0s 809us/step


In [42]:
pd.DataFrame(predictions)

,0,1,2,3,4,5,6,7
0,1.963369,3.603528,3.504807,0.192551,0.997549,0.765607,1.136974,4.725869
1,1.977895,3.594931,3.554045,0.190169,0.998938,0.829842,1.082753,4.730183
2,2.196291,3.635072,3.756991,0.190775,0.990573,1.008635,0.849881,4.766042
3,2.193734,3.618836,3.715956,0.188825,0.984869,0.927535,0.906314,4.740529
4,1.952519,3.752186,3.724901,0.191884,0.974670,0.942191,0.983665,4.730592
...,...,...,...,...,...,...,...,...
4742,1.947174,3.718048,3.623541,0.207094,1.043842,0.984230,0.955593,4.738665
4743,1.945556,3.717829,3.622090,0.206687,1.041010,0.983448,0.953480,4.736945
4744,1.975956,3.755750,3.597001,0.189129,1.014802,0.999667,1.012003,4.733453
4745,2.033715,3.826523,3.617100,0.182851,1.028829,1.059635,1.026737,4.747970


In [43]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, predictions)

0.04086334952684363

### Keras Tuner

In [44]:
import keras_tuner
class AntennaModel(keras_tuner.HyperModel):
    def __init__(self, input_shape):
        self.input_shape= input_shape
        
    def build(self, hp):
        model= keras.Sequential()

        for i in range(hp.Int('num_layers', 1, 4)):
            model.add(keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu', input_shape=self.input_shape))
        model.add(keras.layers.Dense(8)) # Output layer

        # Tune the learning rate for the optimizer 
        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=0.1)

        model.compile(loss='mse',
                    optimizer= keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    metrics= ['mse']
                     )

        return model

In [45]:
import keras_tuner
antenna_model = AntennaModel(input_shape=[len(X_train.keys())])
tuner = keras_tuner.RandomSearch(
    antenna_model,
    objective='val_mse',
    max_trials=3,
    executions_per_trial=1)

tuner.search_space_summary()
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.2, validation_data = (X_test_scaled, y_test), callbacks=[stop_early])

Trial 3 Complete [00h 00m 09s]
val_mse: 0.0441119447350502

Best val_mse So Far: 0.0441119447350502
Total elapsed time: 00h 00m 35s


In [46]:
best_model = tuner.get_best_models(num_models=1)[0]
tuner_preds = best_model.predict(X_test_scaled)
mean_squared_error(y_test, tuner_preds)


149/149 [==============================] - 0s 702us/step


0.04411194081984253

### Sklearn

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

model = MultiOutputRegressor(
    XGBRegressor(n_estimators=1100, min_child_weight=2, max_depth=10, learning_rate=0.2)
)
scaler = StandardScaler() # Initialize scaler
pipeline = Pipeline(steps=[('normalize', scaler), ('model', model)]) # Create pipeline with scaler and model


In [48]:
pipeline.fit(X_train_scaled,y_train)

Pipeline(steps=[('normalize', StandardScaler()),
                ('model',
                 MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             device=None,
                                                             early_stopping_rounds=None,
                                                             enable_categorical=False,
                                                             eval_metric=None,
                                                             feature_types=None,
                                                             gamma=None,
                                                             grow_policy=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.2,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=10,
                                                             max_leaves=None,
                                                             min_child_weight=2,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=1100,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=None, ...)))])

In [49]:
sklearn_pred = pipeline.predict(X_test_scaled)

In [50]:
mean_squared_error(y_test, sklearn_pred)

0.06309396909168913